In [1]:
from langchain_community.utilities import SQLDatabase

In [2]:
host = "localhost"
port = "3306"
username = "root"
password = "root"
database_schema = "text_to_sql"
mysql_url = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"

In [4]:
db = SQLDatabase.from_uri(mysql_url)

print(f"Dialect: {db.dialect}")
print(f"Available Tables: {db.get_table_names()}")

Dialect: mysql
Available Tables: ['budget', 'customers', 'orders', 'products', 'regions', 'state_regions']


In [ ]:
import getpass
import os
os.environ["GEMINI_API_KEY"] = "XYZ"  # Replace 'XYZ' with your actual Google Generative AI API key

from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.5-flash", temperature=0, model_provider="google_genai")

In [6]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [7]:
toolkit.get_tools()

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001DD15504A50>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001DD15504A50>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x000001DD15504A50>),
 QuerySQLCheckerTool(description='Use this tool to 

In [9]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
print(prompt_template.input_variables)

['dialect', 'top_k']


In [10]:
system_message = prompt_template.format(dialect="mysql", top_k=5)

In [13]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, toolkit.get_tools(), prompt=system_message)

In [15]:
example_query = "List the names of customers who have a Total_Unit_Price greater than 6000."

events = agent_executor.stream(
    {"messages": [("user", example_query)]},
    stream_mode = "values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

List the names of customers who have a Total_Unit_Price greater than 6000.
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (6e138e15-b244-4a82-999c-9bd770a09d6a)
 Call ID: 6e138e15-b244-4a82-999c-9bd770a09d6a
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

budget, customers, orders, products, regions, state_regions
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (904b4b48-58ef-4282-98b3-7a40c2a692ac)
 Call ID: 904b4b48-58ef-4282-98b3-7a40c2a692ac
  Args:
    table_names: customers, orders
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE customers (
	`Customer_Name_Index` INTEGER, 
	`Customer_Names` TEXT
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFA

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10
Please retry in 14.305292876s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 14
}
].


================================== Ai Message ==================================
Tool Calls:
  sql_db_query (36fce3fc-332c-4a87-b3b7-6733eae8cbcc)
 Call ID: 36fce3fc-332c-4a87-b3b7-6733eae8cbcc
  Args:
    query: SELECT DISTINCT T1.Customer_Names FROM customers AS T1 JOIN orders AS T2 ON T1.Customer_Name_Index = T2.Customer_Name_Index WHERE T2.Line_Total > 6000 LIMIT 5
================================= Tool Message =================================
Name: sql_db_query

[('Monolith Corp',), ('Krypton Solutions',), ('Blogtags Ltd',), ('Eire Corp',), ('Hydra Company',)]


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 10
Please retry in 10.50766258s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 10
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 10
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<loc

================================== Ai Message ==================================

The names of customers who have a Total_Unit_Price greater than 6000 are: Monolith Corp, Krypton Solutions, Blogtags Ltd, Eire Corp, and Hydra Company.
